In [2]:
#  Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#  Install dependencies
!pip install biopython
!apt-get update -qq && apt-get install -y -qq ncbi-blast+ clustalo

#  Imports
import os
import requests
from io import StringIO
from Bio import SeqIO, pairwise2, AlignIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import ClustalOmegaCommandline

#  parameters
uniprot_id   = "P61626"
drive_folder = "/content/drive/MyDrive/biopython_alignments"
os.makedirs(drive_folder, exist_ok=True)

#  Fetch the query sequence from UniProt
url   = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
resp  = requests.get(url)
query = list(SeqIO.parse(StringIO(resp.text), "fasta"))[0]
SeqIO.write(query, os.path.join(drive_folder, "query.fasta"), "fasta")

#  Run BLASTP against SwissProt
blast_handle = NCBIWWW.qblast(
    program="blastp",
    database="swissprot",
    sequence=query.format("fasta"),
    hitlist_size=16
)
blast_record = NCBIXML.read(blast_handle)

#  Filter out the self-hit and select top 8 non-self accessions
accessions = []
for hit in blast_record.alignments:
    acc = hit.accession
    if acc != uniprot_id and len(accessions) < 8:
        accessions.append(acc)
    if len(accessions) == 8:
        break

#  Download homolog sequences from UniProt
homologs = []
for acc in accessions:
    r = requests.get(f"https://www.uniprot.org/uniprot/{acc}.fasta")
    homologs.extend(list(SeqIO.parse(StringIO(r.text), "fasta")))

# Write combined FASTA (query + homologs)
all_seqs = [query] + homologs
SeqIO.write(all_seqs, os.path.join(drive_folder, "all_seqs.fasta"), "fasta")

#  Pairwise alignments in Clustal (.aln5)
for idx, hit in enumerate(homologs, start=1):
    aln = pairwise2.align.globalxx(query.seq, hit.seq)[0]
    rec_q = SeqRecord(Seq(aln.seqA), id=query.id, description="")
    rec_h = SeqRecord(Seq(aln.seqB), id=hit.id,     description="")
    msa   = MultipleSeqAlignment([rec_q, rec_h])
    out_path = os.path.join(drive_folder, f"pairwise_{idx}.aln5")
    with open(out_path, "w") as handle:
        AlignIO.write(msa, handle, "clustal")

# Multiple sequence alignment with Clustal Omega → multi.aln5
clustal = ClustalOmegaCommandline(
    infile=os.path.join(drive_folder, "all_seqs.fasta"),
    outfile=os.path.join(drive_folder, "multi.aln5"),
    verbose=True,
    auto=True,
    outfmt="clu"
)
clustal()

print(f"All alignments (8 non-self pairs + multi) saved to: {drive_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
All alignments (8 non-self pairs + multi) saved to: /content/drive/MyDrive/biopython_alignments


In [4]:
# Install dependencies
!pip install biopython requests

from Bio.Blast import NCBIWWW, NCBIXML
import requests, re

# 1) Define your mystery sequence
seq = """>mystery_sequence
GATGAPGIAGAPGFPGARGAPGPQGPSGAPGPKXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXGVQGPPGPQGPR
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXGSAGPPGATGFP
GAAGRXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXGVVGLPGQR
"""

# 2) Run BLASTP against NR, filtering organisms that start with "Tyr"
print("Submitting BLASTP… this may take ~1–2 min")
blast_handle = NCBIWWW.qblast(
    program="blastp",
    database="nr",
    sequence=seq,
    entrez_query="Tyr*[Organism]"
)

# 3) Parse the top 5 hits
print("Parsing BLAST results…")
record = NCBIXML.read(blast_handle)
hits = record.alignments[:5]
if not hits:
    raise RuntimeError("No hits found for organisms beginning with 'Tyr'.")

print(f"\nTop {len(hits)} homologs:")
for i, aln in enumerate(hits, 1):
    acc   = aln.accession
    desc  = aln.hit_def
    orgm  = re.search(r"\[([^]]+)\]$", desc)
    org   = orgm.group(1) if orgm else "Unknown"
    e_val = aln.hsps[0].expect
    print(f"{i}. {acc}\n   Desc : {desc}\n   Org  : {org}\n   E-val: {e_val:.2e}\n")

# 4) Fetch the top hit’s UniProt entry and list its literature
top_acc = hits[0].accession
print(f"Fetching literature for top hit ({top_acc}) from UniProt…")

url = f"https://rest.uniprot.org/uniprotkb/{top_acc}.json"
r = requests.get(url)
if r.status_code != 200:
    print("  ⚠️ Could not retrieve UniProt entry.")
else:
    data = r.json()
    refs = data.get("references", [])
    if not refs:
        print("  No references listed in UniProt for this entry.")
    else:
        print(f"\nFound {len(refs)} reference(s):")
        for ref in refs:
            cit = ref.get("citation", {})
            title = cit.get("title", "(no title)")
            # look for a PubMed cross‑ref
            pmid = None
            for db in ref.get("citation", {}).get("dbReference", []):
                if db.get("type") == "PubMed":
                    pmid = db.get("id")
                    break
            pmid_str = f" [PMID:{pmid}]" if pmid else ""
            print(f" - {title}{pmid_str}")


Submitting BLASTP… this may take ~1–2 min
Parsing BLAST results…

Top 5 homologs:
1. P0C2W2
   Desc : RecName: Full=Collagen alpha-1(I) chain; AltName: Full=Alpha-1 type I collagen [Tyrannosaurus rex]
   Org  : Tyrannosaurus rex
   E-val: 1.06e-06

2. XP_050183960
   Desc : collagen alpha-1(I) chain [Myiozetetes cayanensis]
   Org  : Myiozetetes cayanensis
   E-val: 8.25e-06

3. XP_027761390
   Desc : collagen alpha-1(I) chain [Empidonax traillii]
   Org  : Empidonax traillii
   E-val: 8.25e-06

4. GAB3519333
   Desc : hypothetical protein GCM10027442_40970 [Emticicia fontis]
   Org  : Emticicia fontis
   E-val: 1.66e+00

5. KAJ7419418
   Desc : hypothetical protein BTVI_25637 [Pitangus sulphuratus]
   Org  : Pitangus sulphuratus
   E-val: 2.05e+00

Fetching literature for top hit (P0C2W2) from UniProt…
  No references listed in UniProt for this entry.
